In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
#loads pretrained model for vectorizing sentences, Word2Vec
path = "GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(path, binary=True)

In [5]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
def embedSentence(row):
    #uses nltk to seperate sentence into words omitting punctuation
    tokens = tokenizer.tokenize(row)
    #averages all token vectors
    vector = np.array([0]*300)
    count = 0
###################Need to fix bug with possible zero count
    for i in tokens:
        if i in model: #only adds tokens present in the model
            vector = vector + model.get_vector(i)
            count += 1
    vector = vector/max(1, count)
    return vector

In [6]:
#loads emotion training csv
df = pd.read_csv("emotions.csv")

In [7]:
#establishes x, y as input, ouput
#embeds all x from word to values
X = df["text"].apply(embedSentence)
y = df["label"]

In [15]:
#split x, y into training
X = x
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=0)

In [17]:
#builds tensorflow model
tf_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(300, activation="relu"),
                                tf.keras.layers.Dense(300, activation="relu"),
                                tf.keras.layers.Dense(1, activation="sigmoid")
])

In [21]:
tf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

In [23]:
tf_model.evaluate(X_train, y_train)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).